In [2]:
import socket
import threading

active_clients = []

def client_communication(conn, addr):
    print(f"[CONNECTED] Client at {addr} joined the chat.")
    active_clients.append(conn)

    while True:
        try:
            incoming_msg = conn.recv(1024).decode('utf-8')
            if not incoming_msg:
                break
            print(f"[RECEIVED] {addr}: {incoming_msg}")

            # Relay message to all other connected clients
            for client_conn in active_clients:
                if client_conn != conn:
                    try:
                        client_conn.sendall(f"{addr}: {incoming_msg}".encode('utf-8'))
                    except Exception:
                        active_clients.remove(client_conn)
                        client_conn.close()

        except Exception as error:
            print(f"[ERROR] Communication with {addr} failed: {error}")
            break

    print(f"[DISCONNECTED] Client at {addr} left the chat.")
    active_clients.remove(conn)
    conn.close()

def run_server(host='127.0.0.1', port=65432):
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)  # Allow address reuse
    server.bind((host, port))
    server.listen(5)
    print(f"[STARTED] Server listening on {host}:{port}")

    try:
        while True:
            client_conn, client_addr = server.accept()
            thread = threading.Thread(target=client_communication, args=(client_conn, client_addr))
            thread.daemon = True
            thread.start()
    except KeyboardInterrupt:
        print("\n[SHUTDOWN] Server is shutting down.")
    finally:
        server.close()

if __name__ == "__main__":
    run_server()


PermissionError: [WinError 10013] An attempt was made to access a socket in a way forbidden by its access permissions